In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import metrics